In [31]:
import pandas as pd

In [32]:
pwd


'/Users/mov/GitHub/Spiced/spiced_projects/collab2/Supermarket'

In [33]:

# Use pathlib alternatively later
# Load the dataframe with the timestamp, customer_no, and location columns
filepath='/Users/mov/GitHub/Spiced/spiced_projects/collab2/Supermarket/data/monday.csv'
# Read the CSV file into a dataframe
df = pd.read_csv(filepath,sep=';',parse_dates=True)
#df.set_index('timestamp', inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    4884 non-null   object
 1   customer_no  4884 non-null   int64 
 2   location     4884 non-null   object
dtypes: int64(1), object(2)
memory usage: 114.6+ KB


In [34]:
# Convert the timestamp column to a datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices


In [35]:
df = df.set_index(["timestamp"])

In [36]:
# Sort the dataframe by customer_no and timestamp
df = df.sort_values(by=['customer_no', 'timestamp'])
df.head()

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:05:00,1,checkout
2019-09-02 07:03:00,2,dairy
2019-09-02 07:06:00,2,checkout
2019-09-02 07:04:00,3,dairy


In [37]:
df  = df.groupby("customer_no").resample(rule = "T").last()

In [38]:
df

customer_no  location
customer_no timestamp                                 
1           2019-09-02 07:03:00          1.0     dairy
            2019-09-02 07:04:00          NaN      None
            2019-09-02 07:05:00          1.0  checkout
2           2019-09-02 07:03:00          2.0     dairy
            2019-09-02 07:04:00          NaN      None
...                                      ...       ...
1444        2019-09-02 21:48:00       1444.0    spices
            2019-09-02 21:49:00       1444.0  checkout
1445        2019-09-02 21:49:00       1445.0     dairy
1446        2019-09-02 21:50:00       1446.0     dairy
1447        2019-09-02 21:50:00       1447.0     fruit

[10765 rows x 2 columns]

In [39]:
df = df.fillna(method = "ffill")

In [40]:
df.head()

customer_no  location
customer_no timestamp                                 
1           2019-09-02 07:03:00          1.0     dairy
            2019-09-02 07:04:00          1.0     dairy
            2019-09-02 07:05:00          1.0  checkout
2           2019-09-02 07:03:00          2.0     dairy
            2019-09-02 07:04:00          2.0     dairy

In [41]:
# Create a new column called 'duration' with the time difference between each entry  # Here is a mistake...diff is not working?
# df['duration'] = df.groupby('customer_no')['timestamp'].diff()
df['after'] = df['location'].shift(-1)
df.head(20)

customer_no  location     after
customer_no timestamp                                           
1           2019-09-02 07:03:00          1.0     dairy     dairy
            2019-09-02 07:04:00          1.0     dairy  checkout
            2019-09-02 07:05:00          1.0  checkout     dairy
2           2019-09-02 07:03:00          2.0     dairy     dairy
            2019-09-02 07:04:00          2.0     dairy     dairy
            2019-09-02 07:05:00          2.0     dairy  checkout
            2019-09-02 07:06:00          2.0  checkout     dairy
3           2019-09-02 07:04:00          3.0     dairy     dairy
            2019-09-02 07:05:00          3.0     dairy  checkout
            2019-09-02 07:06:00          3.0  checkout     dairy
4           2019-09-02 07:04:00          4.0     dairy     dairy
            2019-09-02 07:05:00          4.0     dairy     dairy
            2019-09-02 07:06:00          4.0     dairy     dairy
            2019-09-02 07:07:00          4.0     dairy  checkout
            2019-09-02 07:08:00          4.0  checkout    spices
5           2019-09-02 07:04:00          5.0    spices  checkout
            2019-09-02 07:05:00          5.0  checkout    spices
6           2019-09-02 07:04:00          6.0    spices     dairy
            2019-09-02 07:05:00          6.0     dairy     dairy
            2019-09-02 07:06:00          6.0     dairy     dairy

In [42]:
# 

In [43]:
# Create a pivot table to summarize the data by customer and location
# pivot = df.pivot_table(index='customer_no', columns='location', values='duration', aggfunc='sum')
# pivot.head()

In [44]:
# Reorder the columns in the pivot table
# pivot = pivot[['dairy', 'fruit','spices', 'drinks','checkout']]
# pivot.head()

In [45]:
# Calculate the transition frequencies using crosstab

transition_frequencies = pd.crosstab(df['location'], df['after'],normalize=0) # Brut force first row 1,0,0,0,0
transition_frequencies.head()

after,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,0.000000,0.284621,0.160056,0.359777,0.195546
dairy,0.088698,0.744206,0.062375,0.051788,0.052933
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.204928,0.087994,0.051623,0.607743,0.047712
spices,0.144137,0.192182,0.176710,0.097720,0.389251


In [46]:

transition_frequencies.iloc[0,:]=[1,0,0,0,0]
transition_frequencies

after,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.088698,0.744206,0.062375,0.051788,0.052933
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.204928,0.087994,0.051623,0.607743,0.047712
spices,0.144137,0.192182,0.176710,0.097720,0.389251


In [47]:
# Calculate the transition probabilities by normalizing the pivot table
transition_probabilities = transition_frequencies.div(transition_frequencies.sum(axis=1), axis=0)
transition_probabilities.head()

after,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.088698,0.744206,0.062375,0.051788,0.052933
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.204928,0.087994,0.051623,0.607743,0.047712
spices,0.144137,0.192182,0.176710,0.097720,0.389251


In [48]:
# Calculate the transition probabilities by normalizing the pivot table
# transition_probabilities = pivot.div(pivot.sum(axis=1), axis=0)
# transition_probabilities.head()

In [49]:
# Store the transition probabilities in a new dataframe
transition_df = pd.DataFrame(transition_probabilities)
transition_df.head()

after,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.088698,0.744206,0.062375,0.051788,0.052933
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.204928,0.087994,0.051623,0.607743,0.047712
spices,0.144137,0.192182,0.176710,0.097720,0.389251


In [50]:
# Rename the columns and index of the dataframe
transition_df.columns = ['checkout', 'dairy', 'drinks','fruit','spices'] # something wrong in my head...
#transition_df.index.name = 'section'

In [51]:
# Display the transition probabilities dataframe
transition_df.head()
# print(transition_df)

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.088698,0.744206,0.062375,0.051788,0.052933
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.204928,0.087994,0.051623,0.607743,0.047712
spices,0.144137,0.192182,0.176710,0.097720,0.389251


In [52]:
transition_df.info() # check calculaion an NaNs - still not in the correct order..

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, checkout to spices
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   checkout  5 non-null      float64
 1   dairy     5 non-null      float64
 2   drinks    5 non-null      float64
 3   fruit     5 non-null      float64
 4   spices    5 non-null      float64
dtypes: float64(5)
memory usage: 240.0+ bytes


In [53]:
transition_df.values
# something wrong with the values or I dont see them...

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.08869814, 0.74420601, 0.06237482, 0.05178827, 0.05293276],
       [0.20810943, 0.01123596, 0.61064973, 0.09086468, 0.07914021],
       [0.20492765, 0.08799374, 0.051623  , 0.60774345, 0.04771216],
       [0.14413681, 0.19218241, 0.1767101 , 0.09771987, 0.38925081]])

In [54]:
pwd


'/Users/mov/GitHub/Spiced/spiced_projects/collab2/Supermarket'

In [55]:
transition_df

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.088698,0.744206,0.062375,0.051788,0.052933
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.204928,0.087994,0.051623,0.607743,0.047712
spices,0.144137,0.192182,0.176710,0.097720,0.389251


In [56]:
transition_df.to_csv('/Users/mov/GitHub/Spiced/spiced_projects/collab2/Supermarket/trans_prob.csv')